In [32]:
!pip install langchain
!pip install langchain-openai

In [33]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [35]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

## Creating basic chain

In [36]:
## Format LCEL : Lang Chain Expression Language
PROMPT = "tell me a joke about {topic}"
prompt = ChatPromptTemplate.from_template(PROMPT)
chain = (prompt
         |
         model)
response = chain.invoke({"topic":"AI"})
print(response.content)

Why did the AI break up with its computer? 

Because it found someone who could handle its data better!


In [37]:
## Running multiple topics
topics = [{"topic":"AI"}, {"topic":"Machine Learning"}]
responses = chain.map().invoke(topics)


In [38]:
for response in responses:
  print(response.content)
  print('....')
  print("\n")

Why did the AI break up with its computer? 

Because it found out it was seeing someone else... a human!
....


Why did the machine learning algorithm break up with the neural network? 

Because it couldn't handle its emotional baggage!
....




## Basic chains do not have history

In [39]:
prompt = ChatPromptTemplate.from_template("{query}")
basic_chain = (prompt
         |
         model)


In [40]:
response = basic_chain.invoke({"query":"What are the first four colors of rainbow?"})
print(response.content)

The first four colors of a rainbow are red, orange, yellow, and green.


In [41]:
response = basic_chain.invoke({"query":"and the other three?"})
print(response.content)

The other three are: 

1. The Father
2. The Son
3. The Holy Spirit


## Adding Memory to the conversational chain

In [42]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda


In [43]:
# Create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","Act as as helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human","{input}"),
    ]
)

In [44]:
# k=3 stores last 3 conversations between you and chatgpt
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

In [45]:
memory.load_memory_variables({}) # shows the conversation history

{'history': []}

In [46]:
from operator import itemgetter
# creates the conversation chain
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables)
        |
        itemgetter("history")
    )
    |
    prompt
    |
    model
)

In [47]:
user_input  = {"input":"What are the first four colors of rainbow?"}
response = chain.invoke(user_input)
print(response.content)

The first four colors of the rainbow are red, orange, yellow, and green.


In [48]:
## Remember to save the history
memory.save_context(user_input, {"output":response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first four colors of the rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={})]}

In [49]:
user_input  = {"input":"What are the remaining colors?"}
response = chain.invoke(user_input)
print(response.content)

The remaining colors of the rainbow are blue, indigo, and violet.


In [50]:
## Remember to save the history
memory.save_context(user_input, {"output":response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first four colors of the rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What are the remaining colors?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The remaining colors of the rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={})]}